In [ ]:
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


To ensure robust model performance, all datasets were divided into three subsets: 70% for training, 20% for validation, and 10% for testing. A stratified splitting strategy was employed to maintain balanced class distributions of AMPs and non-AMPs across all subsets.(DS1)

In [ ]:
import torch
from transformers import BertModel, BertTokenizer, XLNetModel, XLNetTokenizer, T5EncoderModel, T5Tokenizer
import re
import os
import requests
import gc
from tqdm.auto import tqdm
import pandas as pd
import numpy as np

In [ ]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

# خواندن داده‌ها و برچسب‌ها از فایل‌ها
X_train = pd.read_csv('/content/drive/MyDrive/my_project/LMPred_Embeddings/X_train.csv')
X_val = pd.read_csv('/content/drive/MyDrive/my_project/LMPred_Embeddings/X_val.csv')
X_test = pd.read_csv('/content/drive/MyDrive/my_project/LMPred_Embeddings/X_test.csv')

y_train = pd.read_csv('/content/drive/MyDrive/my_project/LMPred_Embeddings/y_train.csv', header=None).values
y_val = pd.read_csv('/content/drive/MyDrive/my_project/LMPred_Embeddings/y_val.csv', header=None).values
y_test = pd.read_csv('/content/drive/MyDrive/my_project/LMPred_Embeddings/y_test.csv', header=None).values

# ترکیب داده‌ها و برچسب‌ها
X_total = pd.concat([X_train, X_val, X_test], ignore_index=True)
y_total = np.concatenate([y_train, y_val, y_test], axis=0)

# شافل کردن داده‌ها و برچسب‌ها به طور همزمان
X_total, y_total = shuffle(X_total, y_total, random_state=42)

# تقسیم داده‌ها به نسبت‌های مشخص شده
X_train_final, X_temp, y_train_final, y_temp = train_test_split(X_total, y_total, test_size=0.3, random_state=42)
X_val_final, X_test_final, y_val_final, y_test_final = train_test_split(X_temp, y_temp, test_size=1/3, random_state=42)  # 1/3 از 0.3 می‌شود 0.1

# ذخیره داده‌ها و برچسب‌ها در فایل‌های جدید
X_train_final.to_csv('/content/drive/MyDrive/my_project/LMPred_Embeddings/X_train_final.csv', index=False)
pd.DataFrame(y_train_final).to_csv('/content/drive/MyDrive/my_project/LMPred_Embeddings/y_train_final.csv', index=False, header=False)

X_val_final.to_csv('/content/drive/MyDrive/my_project/LMPred_Embeddings/X_val_final.csv', index=False)
pd.DataFrame(y_val_final).to_csv('/content/drive/MyDrive/my_project/LMPred_Embeddings/y_val_final.csv', index=False, header=False)

X_test_final.to_csv('/content/drive/MyDrive/my_project/LMPred_Embeddings/X_test_final.csv', index=False)
pd.DataFrame(y_test_final).to_csv('/content/drive/MyDrive/my_project/LMPred_Embeddings/y_test_final.csv', index=False, header=False)

# بررسی ابعاد فایل‌ها
print(f"X_train_final: {X_train_final.shape}")
print(f"y_train_final: {y_train_final.shape}")
print(f"X_val_final: {X_val_final.shape}")
print(f"y_val_final: {y_val_final.shape}")
print(f"X_test_final: {X_test_final.shape}")
print(f"y_test_final: {y_test_final.shape}")


Feature Vector Extraction,T5-XL,LMPred_Embeddings(DS1)

In [ ]:
!pip install -q transformers
!pip install -q transformers sentencePiece

In [ ]:
import torch
from transformers import BertModel, BertTokenizer, XLNetModel, XLNetTokenizer, T5EncoderModel, T5Tokenizer
import re
import os
import requests
import gc
from tqdm.auto import tqdm
import pandas as pd
import numpy as np

In [ ]:
import os

# اتصال Google Drive به محیط کاری
from google.colab import drive
drive.mount('/content/drive')

# مسیر پوشه جدید در Google Drive
folder_path = '/content/drive/MyDrive/my_project/LMPred_Embeddings'

# بررسی وجود پوشه و ایجاد آن در صورت عدم وجود
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    print(f"Folder '{folder_path}' created successfully.")
else:
    print(f"Folder '{folder_path}' already exists.")


In [ ]:
import os
import numpy as np
import torch
import re
from transformers import BertTokenizer, BertModel, T5Tokenizer, T5EncoderModel, XLNetTokenizer, XLNetModel

class LM_EMBED:
    def __init__(self, language_model, max_len, rare_aa):
        self.lang_model = language_model
        self.max_len = max_len
        self.rare_aa = rare_aa

        # وارد کردن توکنایزر و مدل از Rostlab ProtTrans:
        self.tokenizer, self.model = self._load_model_and_tokenizer()

        # تنظیم دستگاه به GPU در صورت موجود بودن:
        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        self.model = self.model.to(self.device).eval()

    def _load_model_and_tokenizer(self):
        model_map = {
            'BERT-BFD': ('Rostlab/prot_bert_bfd', BertTokenizer, BertModel),
            'BERT': ('Rostlab/prot_bert', BertTokenizer, BertModel),
            'T5-XL-BFD': ('Rostlab/prot_t5_xl_bfd', T5Tokenizer, T5EncoderModel),
            'T5-XL-UNI': ('Rostlab/prot_t5_xl_uniref50', T5Tokenizer, T5EncoderModel),
            'XLNET': ('Rostlab/prot_xlnet', XLNetTokenizer, XLNetModel)
        }

        tokenizer_name, tokenizer_class, model_class = model_map.get(self.lang_model, (None, None, None))
        if tokenizer_name is None or tokenizer_class is None or model_class is None:
            raise ValueError("مدل زبانی پشتیبانی نمی‌شود")

        tokenizer = tokenizer_class.from_pretrained(tokenizer_name, do_lower_case=False)
        model = model_class.from_pretrained(tokenizer_name)

        # برای XLNet، تنظیم mem_len
        if self.lang_model == 'XLNET':
            model.config.mem_len = 512

        return tokenizer, model

    def add_spaces(self, seqs_list):
        return [" ".join(sequence) for sequence in seqs_list]

    def extract_word_embs(self, seq_df, filename):
        # ساختن لیستی از توالی‌ها از دیتافریم:
        seqs_list = seq_df.Sequence.to_list()
        seqs_spaced = self.add_spaces(seqs_list)

        if self.rare_aa:
            seqs_spaced = [re.sub(r"[UZOB]", "X", sequence) for sequence in seqs_spaced]

        ids = self.tokenizer.batch_encode_plus(seqs_spaced, add_special_tokens=True, padding='max_length', max_length=self.max_len)
        input_ids = torch.tensor(ids['input_ids']).to(self.device)
        attention_mask = torch.tensor(ids['attention_mask']).to(self.device)

        # پردازش توالی‌ها به صورت دسته‌ای
        batch_size = 10
        num_batches = (len(input_ids) + batch_size - 1) // batch_size
        all_embeddings = []

        with torch.no_grad():
            for i in range(num_batches):
                start = i * batch_size
                end = min((i + 1) * batch_size, len(input_ids))
                batch_input_ids = input_ids[start:end]
                batch_attention_mask = attention_mask[start:end]

                embeddings = self.model(input_ids=batch_input_ids, attention_mask=batch_attention_mask)[0]
                all_embeddings.append(embeddings.cpu().numpy())

        embedding_res = np.concatenate(all_embeddings, axis=0)
        features = self.extract_features(embedding_res, attention_mask.cpu().numpy())
        padded_arr = self.pad(features)

        # مسیر ذخیره‌سازی امبدینگ
        embedding_folder = "/content/drive/MyDrive/my_project/LMPred_Embeddings"
        os.makedirs(embedding_folder, exist_ok=True)
        full_path = os.path.join(embedding_folder, filename)

        # ذخیره آرایه:
        print("در حال ذخیره امبدینگ‌ها...")
        np.save(full_path, padded_arr)

    def extract_features(self, emb_res, att_msk):
        features = []
        for seq_num in range(len(emb_res)):
            seq_len = (att_msk[seq_num] == 1).sum()
            if self.lang_model in ['BERT-BFD', 'BERT']:
                seq_emd = emb_res[seq_num][1:seq_len-1]
            elif self.lang_model in ['T5-XL-BFD', 'T5-XL-UNI']:
                seq_emd = emb_res[seq_num][:seq_len-1]
            elif self.lang_model == 'XLNET':
                padded_seq_len = len(att_msk[seq_num])
                seq_emd = emb_res[seq_num][padded_seq_len-seq_len:padded_seq_len-2]
            features.append(seq_emd)

        return np.array(features, dtype=object)

    def pad(self, features):
        dim1 = self.max_len - 2  # کاهش دو واحد برای توکن‌های CLS و SEP که از قبل حذف شده‌اند
        dim2 = features[0].shape[1]
        padded_arr = np.zeros((len(features), dim1, dim2))

        for i, feature in enumerate(features):
            padded_arr[i, :feature.shape[0], :feature.shape[1]] = feature

        return padded_arr


In [ ]:
X_val = pd.read_csv('/content/drive/MyDrive/my_project/LMPred_AMP_Prediction/LM_Pred_Dataset/X_val_final.csv')


In [ ]:
X_test = pd.read_csv('/content/drive/MyDrive/my_project/LMPred_AMP_Prediction/X_test_final.csv')


In [ ]:
X_train = pd.read_csv('/content/drive/MyDrive/my_project/LMPred_AMP_Prediction/X_train_final.csv')


In [ ]:
# Specifying the max sequence length in the given dataset (255 for the LMPred Dataset),
# then adding 2 to account for special [CLS, SEP] tokens added by the language models):
max_seq_len = 257
T5XL_UNI_EMBED = LM_EMBED('T5-XL-UNI', max_seq_len, True)

In [ ]:
import pandas as pd
import numpy as np

# بارگذاری داده‌ها از CSV
X_train = pd.read_csv('/content/drive/MyDrive/my_project/LMPred_AMP_Prediction/LM_Pred_Dataset/X_train_final.csv')

# پارامترهای مربوط به دسته‌های داده
batch_size = 1000  # تعداد نمونه‌ها در هر دسته
num_batches = (len(X_train) + batch_size - 1) // batch_size  # تعداد کل دسته‌ها

# مسیر خروجی برای ذخیره امبدینگ‌ها
output_dir = '/content/drive/MyDrive/my_project/LMPred_Embeddings/'

# اطمینان از اینکه آبجکت T5XL_UNI_EMBED به درستی ایجاد شده است
if 'T5XL_UNI_EMBED' not in locals():
    raise ValueError("آبجکت T5XL_UNI_EMBED تعریف نشده است.")

# پردازش دسته به دسته
for i in range(num_batches):
    # تعریف محدوده داده‌های فعلی
    start_index = i * batch_size
    end_index = min(start_index + batch_size, len(X_train))

    # استخراج داده‌های فعلی
    X_train_batch = X_train.iloc[start_index:end_index]

    # مسیر خروجی برای ذخیره امبدینگ‌های دسته فعلی
    output_batch = f'{output_dir}T5XL_UNI_INDEP_X_TRAIN_batch_{i+1}.npy'

    # استخراج و ذخیره امبدینگ‌های دسته فعلی
    T5XL_UNI_EMBED.extract_word_embs(X_train_batch, output_batch)

    print(f"امبدینگ‌های دسته {i+1} با موفقیت استخراج و ذخیره شدند.")

print("تمام امبدینگ‌ها با موفقیت استخراج و ذخیره شدند.")


In [ ]:
# تعریف مسیر خروجی برای ذخیره امبدینگ‌های X_val

output_val = '/content/drive/MyDrive/my_project/LMPred_Embeddings/T5XL_UNI_INDEP_X_val.npy'

# استخراج و ذخیره امبدینگ‌های X_val
T5XL_UNI_EMBED.extract_word_embs(X_val, output_val)

print("امبدینگ‌های X_val با موفقیت استخراج و ذخیره شدند.")


# استخراج و ذخیره امبدینگ‌های X_val
T5XL_UNI_EMBED.extract_word_embs(X_val, output_val)

print("امبدینگ‌های X_val با موفقیت استخراج و ذخیره شدند.")


In [ ]:
# تعریف مسیر خروجی برای ذخیره امبدینگ‌های X_test

output_test = '/content/drive/MyDrive/my_project/LMPred_Embeddings/T5XL_UNI_INDEP_X_TEST.npy'

# استخراج و ذخیره امبدینگ‌های X_test
T5XL_UNI_EMBED.extract_word_embs(X_test, output_test)

print("امبدینگ‌های X_test با موفقیت استخراج و ذخیره شدند.")


# استخراج و ذخیره امبدینگ‌های X_test
T5XL_UNI_EMBED.extract_word_embs(X_test, output_test)

print("امبدینگ‌های X_test با موفقیت استخراج و ذخیره شدند.")


resized_X_val.npy

In [ ]:
import numpy as np

def resize_and_save_data(file_paths, save_paths, target_shape):
    for file_path, save_path in zip(file_paths, save_paths):
        data = np.load(file_path)
        num_samples = data.shape[0]
        resized_data = np.zeros((num_samples, target_shape[0], target_shape[1]), dtype=np.float32)
        for i in range(num_samples):
            sample = data[i]
            # فرض بر این است که هر نمونه ممکن است ابعاد متغیری داشته باشد و باید برش یا پر شود
            resized_data[i, :sample.shape[0], :sample.shape[1]] = sample
        np.save(save_path, resized_data)
        print(f"Saved resized data to {save_path}")

# مسیر فایل‌های اصلی و فایل‌های ذخیره شده برای داده‌های آموزشی
file_paths_X = [
    '/content/drive/MyDrive/my_project/LMPred_Embeddings/T5XL_UNI_INDEP_X_TRAIN_batch_1.npy',
    '/content/drive/MyDrive/my_project/LMPred_Embeddings/T5XL_UNI_INDEP_X_TRAIN_batch_2.npy',
    '/content/drive/MyDrive/my_project/LMPred_Embeddings/T5XL_UNI_INDEP_X_TRAIN_batch_3.npy',
    '/content/drive/MyDrive/my_project/LMPred_Embeddings/T5XL_UNI_INDEP_X_TRAIN_batch_4.npy',
    '/content/drive/MyDrive/my_project/LMPred_Embeddings/T5XL_UNI_INDEP_X_TRAIN_batch_5.npy',
    '/content/drive/MyDrive/my_project/LMPred_Embeddings/T5XL_UNI_INDEP_X_TRAIN_batch_6.npy'
]
save_paths_X = [
    '/content/drive/MyDrive/my_project/LMPred_Embeddings/resized_X_TRAIN_batch_1.npy',
    '/content/drive/MyDrive/my_project/LMPred_Embeddings/resized_X_TRAIN_batch_2.npy',
    '/content/drive/MyDrive/my_project/LMPred_Embeddings/resized_X_TRAIN_batch_3.npy',
    '/content/drive/MyDrive/my_project/LMPred_Embeddings/resized_X_TRAIN_batch_4.npy',
    '/content/drive/MyDrive/my_project/LMPred_Embeddings/resized_X_TRAIN_batch_5.npy',
    '/content/drive/MyDrive/my_project/LMPred_Embeddings/resized_X_TRAIN_batch_6.npy'
]

# مسیر فایل‌های اعتبارسنجی
val_file_path_X = ['/content/drive/MyDrive/my_project/LMPred_Embeddings/T5XL_UNI_INDEP_X_val.npy']
val_save_path_X = ['/content/drive/MyDrive/my_project/LMPred_Embeddings/resized_X_val.npy']

# تغییر اندازه و ذخیره داده‌های آموزشی
resize_and_save_data(file_paths_X, save_paths_X, target_shape=(255, 1024))

# تغییر اندازه و ذخیره داده‌های اعتبارسنجی
resize_and_save_data(val_file_path_X, val_save_path_X, target_shape=(255, 1024))


resized_X_TEST.npy

In [ ]:
import numpy as np

def resize_and_save_data(file_path, save_path, target_shape):
    data = np.load(file_path)
    num_samples = data.shape[0]
    resized_data = np.zeros((num_samples, target_shape[0], target_shape[1]), dtype=np.float32)
    for i in range(num_samples):
        sample = data[i]
        # فرض بر این است که هر نمونه ممکن است ابعاد متغیری داشته باشد و باید برش یا پر شود
        resized_data[i, :sample.shape[0], :sample.shape[1]] = sample
    np.save(save_path, resized_data)
    print(f"Saved resized data to {save_path}")

# مسیر فایل تست
test_file_path_X = '/content/drive/MyDrive/my_project/LMPred_Embeddings/T5XL_UNI_INDEP_X_TEST.npy'

# مسیر ذخیره فایل تست
test_save_path_X = '/content/drive/MyDrive/my_project/LMPred_Embeddings/resized_X_TEST.npy'

# تغییر اندازه و ذخیره داده تست
resize_and_save_data(test_file_path_X, test_save_path_X, target_shape=(255, 1024))


This part of the code is related to training and testing the final model, TrancBiM. The model first includes BiLSTM layers, followed by Transformer and attention mechanisms. These are my final results on the first dataset."


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
import matplotlib.pyplot as plt

# تعریف دیتاست سفارشی
class CustomDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return torch.tensor(self.data[idx], dtype=torch.float32), torch.tensor(self.labels[idx], dtype=torch.long)

# تابع برای بارگذاری داده‌ها به عنوان DataLoader
def load_data_as_dataloader(file_paths, labels, batch_size, percentage=1.0):
    data = []
    for file_path in file_paths:
        data.append(np.load(file_path))
    data = np.concatenate(data, axis=0)

    # استفاده از درصدی از داده‌ها
    num_samples = data.shape[0]
    subset_size = int(num_samples * percentage)
    data = data[:subset_size]
    labels = labels[:subset_size]

    dataset = CustomDataset(data, labels)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    return dataloader

# مدل ترنسفورمر با لایه‌های BiLSTM در PyTorch
class TransformerWithBiLSTM(nn.Module):
    def __init__(self, input_dim, num_classes, num_heads=8, num_layers=2, lstm_hidden_dim=32, dropout_rate=0.3):
        super(TransformerWithBiLSTM, self).__init__()

        # لایه‌های BiLSTM
        self.bilstm = nn.LSTM(input_size=input_dim, hidden_size=lstm_hidden_dim,
                              num_layers=num_layers, batch_first=True, bidirectional=True, dropout=dropout_rate)

        self.attention_layers = nn.ModuleList([
            nn.MultiheadAttention(embed_dim=lstm_hidden_dim * 2, num_heads=num_heads) for _ in range(num_layers)
        ])

        self.layer_norm = nn.LayerNorm(lstm_hidden_dim * 2)
        self.fc1 = nn.Linear(lstm_hidden_dim * 2, 64)  # کاهش ابعاد لایه خطی
        self.dropout = nn.Dropout(dropout_rate)        # تغییر نرخ Dropout
        self.fc2 = nn.Linear(64, num_classes)          # سازگاری با ابعاد جدید

    def forward(self, x):
        # عبور از لایه BiLSTM
        lstm_out, _ = self.bilstm(x)

        # عبور از لایه‌های ترنسفورمر
        lstm_out = lstm_out.transpose(0, 1)  # برای سازگاری با PyTorch
        for attention in self.attention_layers:
            lstm_out, _ = attention(lstm_out, lstm_out, lstm_out)

        lstm_out = lstm_out.transpose(0, 1)  # بازگشت به شکل اولیه
        lstm_out = self.layer_norm(lstm_out)
        lstm_out = torch.mean(lstm_out, dim=1)  # GlobalAveragePooling
        lstm_out = torch.relu(self.fc1(lstm_out))
        lstm_out = self.dropout(lstm_out)       # اعمال Dropout
        lstm_out = self.fc2(lstm_out)           # عدم نیاز به Softmax چون CrossEntropyLoss آن را خودکار انجام می‌دهد
        return lstm_out

# تابع آموزش مدل
def train_transformer_model(train_loader, val_loader, input_dim, num_classes, model_save_path, plot_save_path, epochs=100, batch_size=32):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # ایجاد مدل و انتقال به GPU
    model = TransformerWithBiLSTM(input_dim=input_dim, num_classes=num_classes)
    model.to(device)

    optimizer = optim.AdamW(model.parameters(), lr=0.0001, weight_decay=1e-4)  # استفاده از AdamW
    criterion = nn.CrossEntropyLoss()

    # تنظیم نرخ یادگیری
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.5)

    # لیست‌های ذخیره تاریخچه خطا و دقت
    train_loss_history = []
    val_loss_history = []

    best_val_accuracy = 0.0
    early_stopping_counter = 0  # شمارش برای Early Stopping
    patience = 10  # تعداد ایپوک‌ها برای توقف

    # آموزش مدل
    for epoch in range(epochs):
        model.train()
        total_train_loss = 0
        correct_train = 0
        total_train = 0

        for batch_X, batch_y in train_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total_train += batch_y.size(0)
            correct_train += (predicted == batch_y).sum().item()

        train_accuracy = 100 * correct_train / total_train

        # اعتبارسنجی مدل
        model.eval()
        total_val_loss = 0
        correct_val = 0
        total_val = 0
        with torch.no_grad():
            for val_X, val_y in val_loader:
                val_X, val_y = val_X.to(device), val_y.to(device)
                outputs = model(val_X)
                loss = criterion(outputs, val_y)
                total_val_loss += loss.item()

                _, predicted = torch.max(outputs.data, 1)
                total_val += val_y.size(0)
                correct_val += (predicted == val_y).sum().item()

        val_accuracy = 100 * correct_val / total_val

        train_loss_history.append(total_train_loss / len(train_loader))
        val_loss_history.append(total_val_loss / len(val_loader))

        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {total_train_loss / len(train_loader):.4f}, Val Loss: {total_val_loss / len(val_loader):.4f}, Train Accuracy: {train_accuracy:.2f}%, Val Accuracy: {val_accuracy:.2f}%")

        # ذخیره مدل اگر دقت اعتبارسنجی بهبود یافته باشد
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            torch.save(model.state_dict(), model_save_path)
            print(f"New best model saved with Val Accuracy: {val_accuracy:.2f}%")
            early_stopping_counter = 0  # بازنشانی شمارش
        else:
            early_stopping_counter += 1  # افزایش شمارش

        # بروزرسانی نرخ یادگیری
        scheduler.step(total_val_loss)  # به روز رسانی نرخ یادگیری با توجه به ولیدیشن Loss

        # اگر دقت ولیدیشن بهبود نیافت، آموزش متوقف شود
        if early_stopping_counter >= patience:
            print(f"Early stopping triggered after {epoch + 1} epochs.")
            break

    # ذخیره نمودارها
    plt.figure(figsize=(10, 5))
    plt.plot(train_loss_history, label='Train Loss')
    plt.plot(val_loss_history, label='Val Loss')
    plt.title('Model Performance')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig(plot_save_path)
    plt.close()
    print(f"Plots saved to {plot_save_path}")

# فراخوانی تابع train_transformer_model

# مسیر فایل‌ها
save_paths_X = [
    '/content/drive/MyDrive/LM-RESULT2/resized_X_TRAIN_batch_1.npy',
    '/content/drive/MyDrive/LM-RESULT2/resized_X_TRAIN_batch_2.npy',
    '/content/drive/MyDrive/LM-RESULT2/resized_X_TRAIN_batch_3.npy',
    '/content/drive/MyDrive/LM-RESULT2/resized_X_TRAIN_batch_4.npy',
    '/content/drive/MyDrive/LM-RESULT2/resized_X_TRAIN_batch_5.npy',
    '/content/drive/MyDrive/LM-RESULT2/resized_X_TRAIN_batch_6.npy'
]
val_save_path_X = '/content/drive/MyDrive/LM-RESULT2/resized_X_val.npy'
val_label_csv_path = '/content/drive/MyDrive/LM-RESULT2/y_val_final.csv'

# بارگذاری برچسب‌ها
train_labels = np.loadtxt('/content/drive/MyDrive/LM-RESULT2/y_train_final.csv', delimiter=',', dtype=int)
val_labels = np.loadtxt(val_label_csv_path, delimiter=',', dtype=int)

# آماده‌سازی داده‌ها به عنوان DataLoader برای 100 درصد داده‌ها
train_loader = load_data_as_dataloader(save_paths_X, train_labels, batch_size=32, percentage=1.0)
val_loader = load_data_as_dataloader([val_save_path_X], val_labels, batch_size=32, percentage=1.0)

# تنظیمات مدل و آموزش با 100 ایپوک
input_dim = 1024  # ابعاد ورودی داده‌ها (باید با داده‌ها سازگار باشد)
num_classes = 2   # تعداد کلاس‌ها (برچسب‌ها)

train_transformer_model(
    train_loader,          # داده‌های آموزشی
    val_loader,            # داده‌های اعتبارسنجی
    input_dim=input_dim,   # ابعاد ورودی
    num_classes=num_classes,  # تعداد کلاس‌ها
    model_save_path='/content/drive/MyDrive/LM-RESULT2/best0021_model.pth',  # مسیر ذخیره مدل
    plot_save_path='/content/drive/MyDrive/LM-RESULT2/training0021_plot.png',  # مسیر ذخیره نمودار
    epochs=100,            # تعداد ایپوک‌ها
    batch_size=32          # اندازه بچ
)


In [ ]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, roc_auc_score, matthews_corrcoef, roc_curve
import seaborn as sns
import matplotlib.pyplot as plt
import os

# تعریف دیتاست سفارشی برای تست
class CustomTestDataset(Dataset):
    def __init__(self, data, labels=None):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if self.labels is not None:
            return torch.tensor(self.data[idx], dtype=torch.float32), torch.tensor(self.labels[idx], dtype=torch.long)
        else:
            return torch.tensor(self.data[idx], dtype=torch.float32)

# تابع برای بارگذاری داده‌های تست به عنوان DataLoader
def load_test_data_as_dataloader(file_paths, labels=None, batch_size=32):
    data = []
    for file_path in file_paths:
        data.append(np.load(file_path))
    data = np.concatenate(data, axis=0)
    dataset = CustomTestDataset(data, labels)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    return dataloader

# مدل ترنسفورمر با BiLSTM برای تست
class TransformerWithBiLSTM(nn.Module):
    def __init__(self, input_dim, num_classes, num_heads=8, num_layers=2, lstm_hidden_dim=32, dropout_rate=0.3):
        super(TransformerWithBiLSTM, self).__init__()

        # لایه‌های BiLSTM
        self.bilstm = nn.LSTM(input_size=input_dim, hidden_size=lstm_hidden_dim,
                              num_layers=num_layers, batch_first=True, bidirectional=True, dropout=dropout_rate)

        self.attention_layers = nn.ModuleList([nn.MultiheadAttention(embed_dim=lstm_hidden_dim * 2, num_heads=num_heads) for _ in range(num_layers)])

        self.layer_norm = nn.LayerNorm(lstm_hidden_dim * 2)
        self.fc1 = nn.Linear(lstm_hidden_dim * 2, 64)  # کاهش ابعاد لایه خطی
        self.dropout = nn.Dropout(dropout_rate)        # تغییر نرخ Dropout
        self.fc2 = nn.Linear(64, num_classes)          # سازگاری با ابعاد جدید

    def forward(self, x):
        # عبور از لایه BiLSTM
        lstm_out, _ = self.bilstm(x)

        # عبور از لایه‌های ترنسفورمر
        lstm_out = lstm_out.transpose(0, 1)  # برای سازگاری با PyTorch
        for attention in self.attention_layers:
            lstm_out, _ = attention(lstm_out, lstm_out, lstm_out)

        lstm_out = lstm_out.transpose(0, 1)  # بازگشت به شکل اولیه
        lstm_out = self.layer_norm(lstm_out)
        lstm_out = torch.mean(lstm_out, dim=1)  # GlobalAveragePooling
        lstm_out = torch.relu(self.fc1(lstm_out))
        lstm_out = self.dropout(lstm_out)       # اعمال Dropout
        lstm_out = torch.softmax(self.fc2(lstm_out), dim=-1)
        return lstm_out

# تابع تست مدل
def test_transformer_model(test_loader, model_load_path, input_dim=1024, num_classes=2):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = TransformerWithBiLSTM(input_dim=input_dim, num_classes=num_classes)
    model.to(device)

    # بارگذاری مدل ذخیره‌شده
    model.load_state_dict(torch.load(model_load_path, map_location=device))
    model.eval()

    all_predictions = []
    all_labels = []
    all_probs = []

    with torch.no_grad():
        for data, labels in test_loader:
            data, labels = data.to(device), labels.to(device)
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(outputs.cpu().numpy()[:, 1])  # احتمال برچسب مثبت

    all_probs = np.array(all_probs)
    return np.array(all_labels), np.array(all_predictions), all_probs

# محاسبه و نمایش معیارهای عملکرد
def calculate_metrics(y_true, y_pred_labels, y_pred_probs):
    cm = confusion_matrix(y_true, y_pred_labels)
    tn, fp, fn, tp = cm.ravel()
    sensitivity = tp / (tp + fn) * 100
    specificity = tn / (tn + fp) * 100
    accuracy = accuracy_score(y_true, y_pred_labels) * 100
    f1 = f1_score(y_true, y_pred_labels) * 100
    mcc = matthews_corrcoef(y_true, y_pred_labels) * 100
    auroc = roc_auc_score(y_true, y_pred_probs) * 100
    print(f"Sensitivity:     {sensitivity:.2f}%")
    print(f"Specificity:     {specificity:.2f}%")
    print(f"Accuracy:        {accuracy:.2f}%")
    print(f"F1 Score:        {f1:.2f}%")
    print(f"MCC:             {mcc:.2f}%")
    print(f"auROC:           {auroc:.2f}%")

# نمایش ماتریس کانفیوژن درصدی و ذخیره تصویر
def display_conf_matrix(y_true, y_pred_labels, title, filename):
    cm = confusion_matrix(y_true, y_pred_labels)
    cm_percent = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm_percent, annot=True, fmt='.2f', cmap='Blues',
                xticklabels=['Class 0', 'Class 1'],
                yticklabels=['Class 0', 'Class 1'])
    plt.title(title)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.savefig(filename)
    plt.show()

# رسم نمودار ROC
def plot_roc_curve(y_true, y_pred_probs, title, filename):
    fpr, tpr, thresholds = roc_curve(y_true, y_pred_probs)
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc_score(y_true, y_pred_probs):.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(title)
    plt.legend(loc="lower right")
    plt.savefig(filename)
    plt.show()

# مسیر فایل‌های تست
test_save_paths_X = ['/content/drive/MyDrive/LM-RESULT2/resized_X_TEST.npy']
test_label_csv_path = '/content/drive/MyDrive/LM-RESULT2/y_test_final.csv'

# بارگذاری برچسب‌های تست
y_test = np.loadtxt(test_label_csv_path, delimiter=',', dtype=int)

# آماده‌سازی داده‌ها به عنوان DataLoader برای تست
test_loader = load_test_data_as_dataloader(test_save_paths_X, labels=y_test, batch_size=32)

# مسیر مدل ذخیره شده
model_load_path = '/content/drive/MyDrive/LM-RESULT2/best0021_model.pth'

# تست مدل
y_true, y_pred_labels, predicted_probs = test_transformer_model(test_loader, model_load_path)

# محاسبه معیارها
calculate_metrics(y_true, y_pred_labels, predicted_probs)

# ایجاد دایرکتوری برای ذخیره تصاویر در صورت عدم وجود
output_dir = '/content/drive/MyDrive/my_project/LMPred_AMP_Prediction/Figuresfinal'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# نمایش و ذخیره ماتریس کانفیوژن
output_file_cm = os.path.join(output_dir, 'Transformer_BiLSTM3_Model_CM.png')
display_conf_matrix(y_true, y_pred_labels, 'Transformer BiLSTM Model - Confusion Matrix', output_file_cm)

# نمایش و ذخیره نمودار ROC
output_file_roc = os.path.join(output_dir, 'Transformer_BiLSTM3_Model_ROC.png')
plot_roc_curve(y_true, predicted_probs, 'Transformer BiLSTM Model - ROC Curve', output_file_roc)
